# Building a Mitre Attack Navigator Layer and TTPs Timeline using Azure OpenAI
# Visualizing Cyber Threat Data leveraging LLM

__Notebook Version:__ 0.1 <br>
__Notebook Author:__ [Antonio Formato](https://www.linkedin.com/in/antonioformato/)<br>
__Inspired by:__ [Oleksiy Meletskiy](https://www.linkedin.com/in/alecm/)<br>


__Python Version:__ >=Python 3.8<br>

__Data Source Required:__ None<br>

__GPU Compute Required:__ No<br>

__Python Packages:__ 
- requests
- BeautifulSoup (bs4)
- pandas
- os
- json
- uuid
- BlobServiceClient (azure.storage.blob)
- DefaultAzureCredential (azure.identity)
- IFrame (IPython.display)
- base64
- Image (IPython.display)
- display (IPython.display)
- matplotlib.pyplot

## Prerequisites
**Please do not run the notebook cells all at once**. The cells need to be run sequentially and successfully executed before proceeding with the remainder of the notebook.

Make sure you have all the necessary libraries; use 'pip install' to install any missing ones.

**Notice** To integrate the Mitre Attack Navigator, this notebook uses **Azure Blob Storage**: See paragraphs 5 and 6.

## Overview

Jupyter Notebook is a powerful tool that allows you to create and share documents that contain both code and rich text elements, such as figures, links, equations, etc. This versatility makes it a go-to tool for tasks that require iterative development and data exploration.

One of the fascinating applications of Jupyter Notebook is in the field of Cybersecurity, where it can be used to **visualize cyber threat data**. This involves creating an interactive Mitre Attack Navigator Layer and TTPs (Tactics, Techniques, and Procedures) Timeline using Azure OpenAI.

The process involves several steps, including building a web scraper for blog post data, leveraging Azure OpenAI for TTPs extraction, providing a Mitre Attack Layer JSON file, uploading to Mitre Attack Navigator, displaying the Mitre Attack Navigator Layer, generating an attack timeline with Mermaid, and visualizing the attack path.

<div style="color: Black; background-color: LightGreen; padding: 10px; font-size: 18px; line-height: 1.4;">
    <p>The notebook begins by <u>extracting insights from a blog post</u> using a web scraper. The scraped <u>blog post is then analyzed to extract TTPs</u> related to cybersecurity. The extracted TTPs are then used to <u>create a Mitre Attack Layer JSON file</u>. This file is then uploaded to the <u>Mitre Attack Navigator for visualization</u>. Finally, <u>the attack path is visualized</u> using a Mermaid.js timeline graph.</p>
</div>

This notebook stands as an **example of how Jupyter Notebook, coupled with Azure OpenAI, can be used to visualize complex data such as cyber threat data**. The resulting visualizations not only provide a clear picture of the threat landscape but also help in making informed decisions and strategies to counter the threats.

## Table of Contents

1. **Extracting Insights** - Building a Web Scraper for Blog Post Data
2. **Revealing Content** - Printing the Scraped Blog Post
3. **Analyzing Cyber Threats** - Leveraging Azure OpenAI for TTPs Extraction
4. **Mitre Attack Navigator** Providing Mitre Attack Layer JSON File
5. **Navigating Threats** - Uploading to Mitre Attack Navigator
6. **Visualizing Threat** Landscape - Displaying Mitre Attack Navigator Layer
7. **Unveiling Tactics** - Generating an Attack Timeline with Mermaid

**Initialization**

Please install required python package listed in the description above (pip install packages).

## 1 **Extracting Insights** - Building a Web Scraper for Blog Post Data

This section implements a web scraper to extract text content from a specified blog post URL (infosec writeup). 

Utilizing the requests library and BeautifulSoup (bs4), the code ensures successful retrieval of data before proceeding with the extraction.

Example url: [https://blog.talosintelligence.com/lazarus_new_rats_dlang_and_telegram/](https://blog.talosintelligence.com/lazarus_new_rats_dlang_and_telegram/)

In [1]:
#Web scraper
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_text(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # If the GET request is successful, the status code will be 200
    if response.status_code == 200:
        # Get the content of the response
        page_content = response.content
        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(page_content, "html.parser")
        # Get the text of the soup object
        text = soup.get_text()
        # Return the text
        return text
    else:
        return "Failed to scrape the website"

# Enter site
print("Enter blog post writeup url")
url = input()

text = scrape_text(url)

Enter blog post writeup url


## 2 (Optional) **Revealing Content** - Printing the Scraped Blog Post

Following the web scraping, this section prints the extracted blog post text. It serves as a quick check to verify the successful retrieval and display of the scraped content


In [2]:
#print scraped post
print(text)




Operation Blacksmith: Lazarus targets organizations worldwide using novel Telegram-based malware written in DLang...

## 3 **Analyzing Cyber Threats** - Leveraging Azure OpenAI for TTPs Extraction

Using Azure OpenAI, this section extracts Tactics, Techniques, and Procedures (TTPs) related to cybersecurity from the previously scraped text. 

It leverages the power of Large Language Models to analyze and generate a table summarizing the identified cyber threat intelligence.

<div style="color: Black; background-color: LightGoldenRodYellow; padding: 10px; font-size: 18px; line-height: 1.4;">
    <p>Make sure you have defined a .env file with all the following data:</p>
    <ul style="font-size: 14px;">
        <li>AZURE_OPENAI_KEY</li>
        <li>AZURE_OPENAI_ENDPOINT</li>
        <li>DEPLOYMENT_NAME</li>
    </ul>
</div>

<div style="border: solid; padding: 5pt">
<b>Notes: </b>
<p>Azure OpenAI, through the default prompt in the code, extracts a table with TTPs from the writeup text.</p>
</div>

In [3]:
#Provide TTPs table from article
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv("[YOUR ENV FILE].env")

# Set up OpenAI client
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2023-05-15"

)

#Function to provide TTPs (tactics and techniques) table from the scraped text
def ttp(text):
  response = client.chat.completions.create(
      model=os.getenv("DEPLOYMENT_NAME"), # model = "deployment_name"
      messages=[
          {"role": "system", "content": "You are an AI assistant expert in cybersecurity, threat intelligence, and Mitre attack, assisting Infosec professionals in understanding cyber attacks."},
          {"role": "user", "content": f"With reference to ATT&CK Matrix for Enterprise extract TTPs (tactics and techniques) from text at the end of following prompt. For each techniques try to provide techniqueID, tactic, comment if you can get relevant content from text, producing a table with following columns:  technique, technique ID, tactic, comment. Text to work with: {text}"}
      ]
  )

  return response.choices[0].message.content

ttptable = ttp(text)
print(ttptable)

| Technique                                           | Technique ID | Tactic                | Comment                                                                                                                                                                        |
|----------------------------------------------------|--------------|-----------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Custom Command and Control Protocol                | T1094        | Command and Control   | The threat actor(s) utilized custom command and control protocols via Telegram to take control of the infected systems.                                                        |
| Domain Generation Algorithms                       | T1568        | Command and Control   | The threat actor(s) used DGA to evade detection and hide their command and control servers. 

## 4 **Mitre Attack Navigator** Providing Mitre Attack Layer JSON File
This section focuses on the creation and provision of a Mitre Attack Layer JSON file, incorporating Tactics, Techniques, and Procedures (TTPs) identified in a cyber threat scenario. 

Leveraging Azure OpenAI capabilities, the code generates a JSON file formatted for MITRE ATT&CK Navigator, facilitating the visualization of cyber threat data. The included TTPs are extracted from the provided prompt table, resulting in a detailed and actionable representation of the threat landscape.

Note: Ensure that the provided '_ttptable_' variable contains valid Tactics, Techniques, and Procedures extracted from the cyber threat intelligence analysis section.


<div style="border: solid; padding: 5pt">
<b>Notes: </b>
<p>The prompt uses the one-shot example technique with the information provided by variables 'prompt_response' and 'prompt_table', which you can read in the code below. Feel free to modify to experiment with different approaches..</p>
</div>

In [4]:
#provide Mitre Attack layer json file
import os
import json
from openai import AzureOpenAI
from uuid import uuid4
from dotenv import load_dotenv

load_dotenv("[YOUR ENV FILE].env")

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2023-05-15"
)

prompt_table = """
| Technique                                | Technique ID | Tactic           | Comment                                                                                                      |
|------------------------------------------|--------------|------------------|--------------------------------------------------------------------------------------------------------------|
| Used CVE-2021-44228 for initial access.   | T1190        | Initial Access   | Used CVE-2021-44228 to exploit publicly exposed servers for initial access.                                  |
| Used commands and scripts for execution.  | T1059        | Execution        | Used commands and scripts (like PowerShell and BAT) to execute different operations.                         |
| Used NineRAT for persistence.             | T1543        | Persistence      | Used NineRAT to set up persistence by creating services using BAT scripts.                                     |
| NineRAT dropper deletes itself for defense evasion. | T1140 | Defense Evasion  | NineRAT has a dropper binary containing two other components, written to disk, and the dropper deletes itself to avoid detection. |
| Used Telegram for command and control.   | T1102        | Command and Control | Used Telegram bots and channels for C2 communications.                                                       |
| Used commands for system information discovery. | T1082 | Discovery        | Used commands like "whoami," "ver," "getmac" for system information discovery.                                |
| Used NineRAT for data collection.         | T1005        | Collection       | NineRAT is used to collect data from the local system.                                                        |
"""

prompt_response = """
{
  "name": "Lazarus Group TTPs",
  "versions": {
    "attack": "14",
    "navigator": "4.9.1",
    "layer": "4.5"
  },
  "domain": "enterprise-attack",
  "description": "TTPs identified in Lazarus Group's Operation Blacksmith",
  "filters": {
    "platforms": ["windows"]
  },
  "sorting": 0,
  "layout": {
    "layout": "side",
    "aggregateFunction": "average",
    "showID": false,
    "showName": true,
    "showAggregateScores": false,
    "countUnscored": false,
    "expandedSubtechniques": "none"
  },
  "hideDisabled": false,
  "techniques": [
    {
      "techniqueID": "T1190",
      "tactic": "initial-access",
      "color": "",
      "comment": "Used CVE-2021-44228 to exploit publicly exposed servers for initial access.",
      "enabled": true,
      "metadata": [],
      "links": [],
      "showSubtechniques": false
    },
    {
      "techniqueID": "T1059",
      "tactic": "execution",
      "color": "",
      "comment": "Used commands and scripts (like PowerShell and BAT) to execute different operations",
      "enabled": true,
      "metadata": [],
      "links": [],
      "showSubtechniques": false
    },
    {
      "techniqueID": "T1543",
      "tactic": "persistence",
      "color": "",
      "comment": "Used NineRAT to set up persistence by creating services using BAT scripts.",
      "enabled": true,
      "metadata": [],
      "links": [],
      "showSubtechniques": false
    },
    {
      "techniqueID": "T1140",
      "tactic": "defense-evasion",
      "color": "",
      "comment": "NineRAT has a dropper binary containing two other components, which are written to disk and the dropper deletes itself to avoid detection.",
      "enabled": true,
      "metadata": [],
      "links": [],
      "showSubtechniques": false
    },
    {
      "techniqueID": "T1102",
      "tactic": "command-and-control",
      "color": "",
      "comment": "Used Telegram bots and channels for C2 communications.",
      "enabled": true,
      "metadata": [],
      "links": [],
      "showSubtechniques": false
    },
    {
      "techniqueID": "T1082",
      "tactic": "discovery",
      "color": "",
      "comment": "Used commands like \"whoami\", \"ver\", \"getmac\" for system information discovery.",
      "enabled": true,
      "metadata": [],
      "links": [],
      "showSubtechniques": false
    },
    {
      "techniqueID": "T1005",
      "tactic": "collection",
      "color": "",
      "comment": "NineRAT is used to collect data from the local system.",
      "enabled": true,
      "metadata": [],
      "links": [],
      "showSubtechniques": false
    }
  ],
  "gradient": {
    "colors": ["#ff6666", "#ffe766", "#8ec843"],
    "minValue": 0,
    "maxValue": 100
  },
  "legendItems": [],
  "metadata": [],
  "links": [],
  "showTacticRowBackground": false,
  "tacticRowBackground": "#dddddd"
}
"""

#Function to provide ATT&CK Matrix for Enterprise layer json file
def attack_layer(ttptable):
  response = client.chat.completions.create(
      model=os.getenv("DEPLOYMENT_NAME"), # model = "deployment_name" 
      messages=[
                  {
                      "role": "system",
                      "content": f"You are tasked with creating an ATT&CK Matrix for Enterprise layer json file with attack version 14, navigator 4.9.1, layer version 4.5 to load a layer in MITRE ATT&CK Navigator. Use {ttptable} as input. Print just json content, avoiding ncluding any additional text in the response. In domain filed use enterprise-attack."
                  },
                  {
                      "role":"user",
                      "content":f"Title:  Enterprise techniques used by 2015 Ukraine Electric Power Attack, ATT&CK campaign C0028 (v1.0): Table: {prompt_table}"
                  },
                  {
                      "role":"assistant",
                      "content":f"{prompt_response}"
                  }
              ]
  )
  return response.choices[0].message.content

mitre_layer = attack_layer(ttptable)
print(mitre_layer)

# Generate a unique ID for the file name
unique_id = str(uuid4())

# Define the file path with a unique ID
file_path = os.path.join("mitrelayers", f"mitre-attack-navigator-{unique_id}.json")

# Write ttptable content to the file
with open(file_path, 'w') as json_file:
    json_file.write(mitre_layer)

# Print the file path for reference
print(f"Content written to: {file_path}")

{
  "versions": {
    "navigator": "4.9.1",
    "layer": "4.5",
    "attack": "14"
  },
  "name": "Lazarus_Group_TTPs",
  "domain": "enterprise-attack",
  "techniques": [
    {
      "techniqueID": "T1094",
      "tactic": "command-and-control",
      "comment": "The threat actor(s) utilized custom command and control protocols via Telegram to take control of the infected systems.",
      "enabled": true
    },
    {
      "techniqueID": "T1568",
      "tactic": "command-and-control",
      "comment": "The threat actor(s) used DGA to evade detection and hide their command and control servers.",
      "enabled": true
    },
    {
      "techniqueID": "T1003",
      "tactic": "credential-access",
      "comment": "The threat actor(s) used built-in Windows utilities to dump credentials.",
      "enabled": true
    },
    {
      "techniqueID": "T1219",
      "tactic": "command-and-control",
      "comment": "RATs \"NineRAT\" and \"DLRAT\" were used by Lazarus group to get control over the

## 5 **Navigating Threats** - Uploading to Mitre Attack Navigator

In this part, the code handles the integration of cyber threat intelligence by uploading the generated MITRE ATT&CK® Navigator Layer to Azure Blob Storage. 
This step is reuired for sharing and visualizing threat data across different platforms.

The ATT&CK Navigator is a web-based tool for annotating and exploring ATT&CK matrices. It can be used to visualize defensive coverage, red/blue team planning, the frequency of detected techniques, and more. 

The most effective and fast way I found to visualize the MITRE ATT&CK® Navigator Layer in the notebook was to publish the JSON file (representing the Mitre layer) to Azure Storage Account and load the layer directly onto the online tool [https://mitre-attack.github.io/attack-navigator/](https://mitre-attack.github.io/attack-navigator/). 

<div style="border: solid; padding: 5pt">
<b>Notes: </b>
<p>It is recommended to modify the connection parameters of your Storage Account.</p>
</div>

<div style="color: Black; background-color: LightGoldenRodYellow; padding: 10px; font-size: 18px; line-height: 1.4;">
    <p>Make sure you have defined a .env file with all the following data:</p>
    <ul style="font-size: 14px;">
        <li>AZURE_CLIENT_ID</li>
        <li>AZURE_TENANT_ID</li>
        <li>AZURE_CLIENT_SECRET</li>
    </ul>
</div>

In [5]:
#Upload of the previously defined Mitre Attack Navigator layer. 
from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os

print("Loading environment variables from .env file")
load_dotenv("[YOUR ENV FILE].env")

# Create a credential object
credential = DefaultAzureCredential()

# Specify the URL of the Azure Storage account
account_url = "https://YOURBLOBURL.blob.core.windows.net/"

# Create a blob service client using account_url
blob_service_client = BlobServiceClient(account_url=account_url, credential=credential)

# Upload the file to a container named "layers"
# Blob Storage container name
container_name = "layers"

# Blob name and local file path
blob_name = f"mitre-attack-navigator-{unique_id}.json"
local_file_path = os.path.join("mitrelayers", f"mitre-attack-navigator-{unique_id}.json")
print(blob_name)

try:
    # Upload the file
    with open(local_file_path, "rb") as f:
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
        blob_client.upload_blob(f.read())

    # Check if the file has been uploaded
    exists = blob_client.exists()
    print(f"File {blob_name} uploaded successfully: {exists}")

except Exception as e:
    # Print the exception details for debugging
    print(f"Error: {e}")
    raise e

Loading environment variables from .env file
mitre-attack-navigator-664f28d5-c70b-439f-bf60-92d49a218dca.json
File mitre-attack-navigator-664f28d5-c70b-439f-bf60-92d49a218dca.json uploaded successfully: True


## 6 **Visualizing Threat Landscape** - Displaying Mitre Attack Navigator Layer

This section employs the IPython IFrame to display the uploaded MITRE ATT&CK® Navigator Layer interactively. 

It provides a user-friendly interface for exploring and understanding the identified threat landscape.


In [6]:
from IPython.display import IFrame
print(f"mitre-attack-navigator-{unique_id}.json")
# The iframe URL
iframe_url = (
    "https://mitre-attack.github.io/attack-navigator/"
    "#layerURL=https%3A%2F%2Fmitrenavigator.blob.core.windows.net"
    f"%2Flayers%2F{blob_name}"
)

# Create the IFrame
iframe = IFrame(src=iframe_url, width="100%", height=600)

# Display the IFrame
iframe

mitre-attack-navigator-664f28d5-c70b-439f-bf60-92d49a218dca.json


## 7 **Visualization of the Attack Path** - Generating an Attack Timeline with Mermaid

This section is dedicated to visualizing the attack path of a cyber threat. Leveraging Azure OpenAI, the code generates a list of Tactics and Subtactics ordered by execution time based on the provided Threat Intelligence and Tactics, Techniques, and Procedures (TTPs) table. The output is a clear and structured representation of the chronological sequence of cyber threat actions.

Following this, the code uses Mermaid.js to create an illustrative timeline graph of the attack stages. The timeline graph is constructed based on the provided TTPs timeline and Mermaid.js code, offering a visual narrative of the key events and tactics employed during the cyber attack. This visualization aids in understanding the progression of the attack, providing valuable insights for threat analysis and mitigation.

<div style="border: solid; padding: 5pt">
<b>Notes: </b>
<p>Ensure that the provided 'text' and 'ttptable' variables contain valid data related to the cyber threat intelligence analysis.</p>
</div>

The Mermaid.js timeline code generated serves as a visual representation of the attack path and should be used within a compatible environment or viewer that supports Mermaid.js syntax.

In [7]:
#Visualization of the attack path
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv("[YOUR ENV FILE].env")

# Set up OpenAI client
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2023-05-15"

)

#Function to provide a list of TTPs order by execution time
def ttp_list(text, ttptable):
    response = client.chat.completions.create(
        model=os.getenv("DEPLOYMENT_NAME"), # model = "deployment_name"
        messages=[
            {"role": "system", "content": "You are an AI assistant expert in cybersecurity, threat intelligence, and Mitre attack, assisting Infosec professionals in understanding cyber attacks."},
            {"role": "user", "content": f"Based on {text} and {ttptable} provide a list of TTPs order by execution time, Each line must include only Tactic and Subtactic, IDs between brackets after subtactic."}
        ]
    )

    return response.choices[0].message.content

attackpath = ttp_list(text, ttptable)
print(attackpath)

ttps_timeline = """
1. Initial Access: Exploitation of Remote Services [T1210]
2. Execution: Command and Scripting Interpreter: PowerShell [T1059.001]  
3. Persistence: External Remote Services [T1133]
4. Persistence: Server Software Component: Web Shell [T1505.003]
5. Persistence: Account Creation [T1136]
6. Defense Evasion: Use Alternate Authentication Material [T1550]
7. Defense Evasion: Modify Registry [T1112]
8. Defense Evasion: Indicator Removal on Host: File Deletion [T1070.004]
9. Credential Access: OS Credential Dumping [T1003]
10. Discovery: System Information Discovery [T1082]
11. Collection: Data Staged: Local Data Staging [T1074.001]
12. Command and Control: Remote Access Tools [-]
13. Command and Control: Proxy: Multi-hop Proxy [T1090.003]
14. Command and Control: Application Layer Protocol: Web Protocols [T1071.001]
15. Command and Control: Ingress Tool Transfer [T1105]
16. Impact: Data Encrypted for Impact [T1486]
"""

mermaid_timeline = """
timeline
title Lazarus Group Operation Blacksmith
    Initial Access
    : Exploitation of Remote Services - [T1210]
	Execution
    : Command and Scripting Interpreter - PowerShell - [T1059.001]  
    Persistence
    : External Remote Services - [T1133]
	: Server Software Component - Web Shell - [T1505.003]
	: Account Creation - [T1136]
	Defense Evasion
	: Use Alternate Authentication Material - [T1550]
	: Modify Registry - [T1112]
	: Indicator Removal on Host - File Deletion - [T1070.004]
	Credential Access
    : OS Credential Dumping - [T1003]
    Discovery
    : System Information Discovery - [T1082]
    Collection: Data Staged - Local Data Staging - [T1074.001]
	Command and Control
    : Remote Access Tools - [-]
    : Proxy Multi-hop Proxy - [T1090.003]
    : Application Layer Protocol Web Protocols - [T1071.001]
	: Ingress Tool Transfer - [T1105]
    Impact: Data Encrypted for Impact - [T1486]
"""

#Function to generate attack timeline Mermaid.jc code 
def ttp_graph_timeline(ttps_timeline, mermaid_timeline):
    response = client.chat.completions.create(
        model="gpt4-af-demo", 
        messages=[
                    {
                        "role":"user",
                        "content":f"Write a Mermaid.js timeline graph that illustrates the stages of a cyber attack. As an example condider the Lazarus Group's operation named Operation Blacksmith, whose Tactics, Techniques, and Procedures (TTPs) timeline is as follows: {ttps_timeline}, and related meirmad.js code is: {mermaid_timeline}. Use the following guidalines to generate code: \n1. timeline: This keyword starts the timeline graph definition, \n2. title: This keyword is followed by the title of the timeline graph, \n3. Each timeline step is defined on a separate line and starts with a description of the step. The description should be concise and informative, summarizing the key actions or events of the step, \n4. The description is followed by a colon (:) and then the step details, \n5. The step details can include any additional information about the step, such as the specific tools or techniques used. \n6. Optionally, the step details can include a reference to a malleable threat technique ID using square brackets. \n7 avoid provide days after TTP ID. \n8 Provide just mermaid.js code without any other text. \n9 start code with timeline- \n10 don't use any bracket at the benning and the end of your output "
                    }
                ]    
    )

    return response.choices[0].message.content

mermaid_timeline = ttp_graph_timeline(ttps_timeline, mermaid_timeline)
print(mermaid_timeline)


1. [T1190] Initial Access: Exploit Public-Facing Application
2. [T1082] Discovery: System Information Discovery
3. [T1083] Discovery: File and Directory Discovery
4. [T1136] Persistence: Account Creation
5. [T1061] Execution: Graphical User Interface
6. [T1018] Discovery: Remote System Discovery
7. [T1059.001] Execution: Command and Scripting Interpreter: PowerShell
8. [T1094] Command and Control: Custom Command and Control Protocol
9. [T1550.002] Defense Evasion: Use Alternate Authentication Material: Token
10. [T1132] Exfiltration: Data Encoding
11. [T1562.001] Defense Evasion: Impair Defenses: Disable or Modify Tools
12. [T1547.009] Persistence: Boot or Logon Autostart Execution: Shortcut Modification
13. [T1112] Defense Evasion: Modify Registry
14. [T1071] Command and Control: Application Layer Protocol
timeline
title Operation Blacksmith
    Initial Access: Exploitation of Remote Services [T1210]
	Execution: Command and Scripting Interpreter - PowerShell [T1059.001]  
	Persistence

## 8 **Cyber Attack Timeline Visualization** using markdown tool

This section utilizes Mermaid.js to create an interactive and visually appealing timeline graph representing the stages of a cyber attack. Mermaid is a markdown tool that renders text into diagrams

The code takes the generated Mermaid.js timeline code as input and converts it into a base64-encoded image. The resulting image is then displayed using IPython's Image module.

The purpose of this code is to provide an interactive and graphical representation of the cyber attack timeline, enhancing the understanding of the chronological sequence of events, tactics, and techniques involved in the cyber threat scenario. The visualization aids cybersecurity professionals and analysts in comprehending the flow of the attack, facilitating more effective threat analysis and response.

<div style="border: solid; padding: 5pt">
<b>Notes: </b>
<p>Ensure that the mermaid_timeline variable contains valid Mermaid.js code generated from the previous sections of the notebook. The code is designed to be executed in a Jupyter Notebook environment, leveraging the capabilities of IPython to display the generated timeline image.</p>
</div>

In [8]:
#Mermaid.js cyber attack timeline
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

def generate_timeline(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

generate_timeline(mermaid_timeline)

The features of this Jupyter Notebook will soon be added to the **TI Mindmap** tool. 
If you want to learn more:
- Tool: [https://ti-mindmap-gpt.streamlit.app/](https://ti-mindmap-gpt.streamlit.app/)
- GitHub: [https://github.com/format81/TI-Mindmap-GPT](https://github.com/format81/TI-Mindmap-GPT)
- Blog posts:
    - [Introducing TI Mindmap GPT](https://medium.com/@antonio.formato/introducing-ti-mindmap-gpt-6f433f140488)
    - [Enhancing Cyber Threat Intelligence with TI Mindmap GPT: Integration of Azure OpenAI and advanced features](https://medium.com/microsoftazure/enhancing-cyber-threat-intelligence-with-ti-mindmap-gpt-integration-of-azure-openai-and-advanced-94121ed66ac4)

<div style="color: Black; background-color: LightGreen; padding: 10px; font-size: 18px; line-height: 1.4;">
    <p>I hope you find this interesting. If you’ve found this Notebook useful, I invite you to contribute and add a star on <a href="https://github.com/format81/mitre-threat-visualization-azure-openai/" style="color: #0077b5; text-decoration: underline;" target="_blank">GitHub</a> and follow me on <a href="https://www.linkedin.com/in/antonioformato/" style="color: #0077b5; text-decoration: underline;" target="_blank">LinkedIn</a>.</p>
</div>